In [1]:
from functools import partial

import os.path as osp
from torch_geometric.datasets import Planetoid,PPI

import torch
import torch.nn as nn
import time
from modeling.models import TransformerModel
from modeling.data import bert_walk_collate, BertWalkDataset
from modeling.tokenizer import bert_walk_tokenizer
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from utils.commons import *
import random
import argparse
# from train_bert_walk import *
# bert_walk_tokenizer, bert_walk_collate

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(flush_secs=10)


2024-12-02 15:27:46.327181: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-02 15:27:46.449154: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-02 15:27:46.937297: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-12-02 15:27:46.937344: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [2]:
# device = "cpu"

device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
import torch 

checkpoint = torch.load("/home/netra-mobile/Desktop/GRAPH_BERT_VAE/artifacts/Dec02_11-42-04_netramobile-precision-7560_model.pt", map_location=torch.device('cpu'))

In [4]:
checkpoint.keys()

dict_keys(['model_params', 'model_state_dict', 'networks', 'tokenizer'])

In [5]:
checkpoint['model_params']

{'batch_size': 64,
 'emsize': 128,
 'nhid': 200,
 'nlayers': 6,
 'nhead': 4,
 'dropout': 0.0,
 'learning_rate': 0.0005,
 'epochs': 40,
 'K': 1,
 'alpha': 0.1,
 'mask_rate': 0.2,
 'q': 1,
 'p': 1,
 'walk_length': 10,
 'num_walks': 10,
 'weighted': 1,
 'directed': 0,
 'organism': 'FIRSTMM_DB',
 'ntokens': 3533}

In [6]:
graphs = ['/home/netra-mobile/Desktop/GRAPH_BERT_VAE/inputs/FIRSTMM_DB/FIRSTMM_DB_11.txt']

In [7]:
data, pyg_graphs = read_data(graphs, checkpoint['model_params'])


['/home/netra-mobile/Desktop/GRAPH_BERT_VAE/inputs/FIRSTMM_DB/FIRSTMM_DB_11.txt']

Reading Corpus from Disk


In [8]:
pyg_graphs

[Data(edge_index=[2, 6014], num_nodes=1304, edge_weight=[6014], node_index=[1304], node_sequence=[1])]

In [9]:
pyg_graphs[0].num_nodes

1304

In [10]:
checkpoint['model_params']['epochs'] = 20
checkpoint['model_params']['ntokens'] = pyg_graphs[0].num_nodes

checkpoint

{'model_params': {'batch_size': 64,
  'emsize': 128,
  'nhid': 200,
  'nlayers': 6,
  'nhead': 4,
  'dropout': 0.0,
  'learning_rate': 0.0005,
  'epochs': 20,
  'K': 1,
  'alpha': 0.1,
  'mask_rate': 0.2,
  'q': 1,
  'p': 1,
  'walk_length': 10,
  'num_walks': 10,
  'weighted': 1,
  'directed': 0,
  'organism': 'FIRSTMM_DB',
  'ntokens': 1304},
 'model_state_dict': OrderedDict([('pos_encoder.pe',
               tensor([[[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
                          0.0000e+00,  1.0000e+00]],
               
                       [[ 8.4147e-01,  5.4030e-01,  7.6172e-01,  ...,  1.0000e+00,
                          1.1548e-04,  1.0000e+00]],
               
                       [[ 9.0930e-01, -4.1615e-01,  9.8705e-01,  ...,  1.0000e+00,
                          2.3096e-04,  1.0000e+00]],
               
                       ...,
               
                       [[ 6.1950e-02,  9.9808e-01,  8.1507e-01,  ...,  9.9770e-01,
                 

In [11]:
from tokenizers.pre_tokenizers import WhitespaceSplit
from tokenizers import Tokenizer, models, trainers
from pathlib import Path


def bert_walk_tokenizer(data, model_params):
    tokenizer = Tokenizer(models.WordLevel(unk_token="[UNK]"))
    #my_tokens = Path("tokenizer_test_.pt")
    
    print("training tokenizer!")
    tokenizer.pre_tokenizer = WhitespaceSplit()
    trainer = trainers.WordLevelTrainer(special_tokens=["[UNK]", "[MASK]", "[CLS]", "[PAD]"])
    tokenizer.enable_padding()
    tokenizer.train_from_iterator(map(lambda x: x.split(), data), trainer=trainer)
    tokenizer.save('tokenizer_test_.pt')
    return tokenizer

In [12]:
tokenizer = bert_walk_tokenizer(data, checkpoint['model_params'])

training tokenizer!


In [13]:
pyg_graphs

[Data(edge_index=[2, 6014], num_nodes=1304, edge_weight=[6014], node_index=[1304], node_sequence=[1])]

In [14]:
tokenizer.get_vocab_size(), len(torch.tensor(tokenizer.encode(pyg_graphs[0]["node_sequence"][0]).ids))

(2945, 1304)

In [15]:
for net in pyg_graphs:
        net["node_tokens"] = torch.tensor(tokenizer.encode(net["node_sequence"][0]).ids).to(device)
        net.edge_index = net.edge_index.to(device)
        net.edge_weight = net.edge_weight.to(device)
checkpoint["model_params"]["ntokens"] = tokenizer.get_vocab_size()

In [16]:
checkpoint["model_params"]

{'batch_size': 64,
 'emsize': 128,
 'nhid': 200,
 'nlayers': 6,
 'nhead': 4,
 'dropout': 0.0,
 'learning_rate': 0.0005,
 'epochs': 20,
 'K': 1,
 'alpha': 0.1,
 'mask_rate': 0.2,
 'q': 1,
 'p': 1,
 'walk_length': 10,
 'num_walks': 10,
 'weighted': 1,
 'directed': 0,
 'organism': 'FIRSTMM_DB',
 'ntokens': 2945}

In [17]:
model = TransformerModel(checkpoint['model_params'], pyg_graphs).to(device)
dataset = BertWalkDataset(data, tokenizer)
dataloader = DataLoader(
        dataset,
        batch_size=checkpoint['model_params']["batch_size"],
        collate_fn=partial(bert_walk_collate, tokenizer=tokenizer),
        shuffle=True,
    )

In [18]:
model

TransformerModel(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=200, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (linear2): Linear(in_features=200, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.0, inplace=False)
        (dropout2): Dropout(p=0.0, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        

In [19]:
def train_bert_walk(model, dataloader, model_params, tokenizer):
    lr = model_params["learning_rate"]
    mask_token_id = tokenizer.get_vocab()["[MASK]"]
    model.train()
    criterion = nn.CrossEntropyLoss(ignore_index=-100)
    optim = torch.optim.Adam(model.parameters(), lr=lr)
    best_loss = float("inf")
    best_model = None
    for epoch in range(model_params["epochs"]):
        total_loss = 0
        epoch_start_time = time.time()
        model.all_prop_emb()
        for b, batch in enumerate(dataloader):
            optim.zero_grad()
            input = batch["input"].clone()
            labels = batch["input"].clone()
            src_mask = batch["src_mask"]
            src_key_padding_mask = batch["src_key_padding_mask"]

            rand_mask = ~batch["input"].bool()
            for i, row in enumerate(
                torch.randint(
                    1,
                    batch["input"].shape[1],
                    (batch["input"].shape[0], int(batch["input"].shape[1] * model_params["mask_rate"])),
                )
            ):
                rand_mask[i, row] = True

            mask_idx = (rand_mask.flatten() == True).nonzero().view(-1)
            input = input.flatten()
            input[mask_idx] = mask_token_id
            input = input.view(batch["input"].size())
            labels[input != mask_token_id] = -100

            out = model(input.to(device), src_mask.to(device), src_key_padding_mask.to(device))
            loss = criterion(out.view(-1, model_params["ntokens"]), labels.view(-1).to(device))
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            optim.step()

            total_loss += loss.item()

        elapsed = time.time() - epoch_start_time
        loss_dict = {}
        val_loss = total_loss / (len(dataloader))
        loss_dict["total"] = val_loss
        
        print("-" * 89)
        print(f"| epoch {1+epoch:3d} | time: {elapsed:5.2f}s | " f"loss {val_loss:5.2f} | lr {lr:02.5f}")
        print("-" * 89)


        
        if total_loss < best_loss:
            best_loss = total_loss
            best_model = model.state_dict()
        

    return best_model


In [ ]:
mdl = train_bert_walk(model, dataloader, checkpoint['model_params'], tokenizer)

In [ ]:
torch.save(
        {
            "model_params": checkpoint['model_params'],
            "model_state_dict": mdl,
            "networks": pyg_graphs,
            "tokenizer": tokenizer,
        },
        "test_model.pt",
    )